## overview

In [1]:
# default package
import logging
import sys 
import os
import pathlib
import IPython
import random

In [2]:
# third party package
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import psycopg2
import sqlalchemy
from sqlalchemy.sql.expression import column,select,table
from sqlalchemy import create_engine
import sklearn.datasets

In [3]:
# my package
sys.path.append(os.path.join(pathlib.Path().resolve(),"../"))

In [4]:
# reload settings
%load_ext autoreload
%autoreload 2

In [5]:
# logger
logger=logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [6]:
## grap setting
sns.set()

In [7]:
## db setting
engine = create_engine('postgresql://postgres:postgres@db:5432/postgres')
with engine.connect() as conn:
    print(conn.closed)

False


## sql to python

In [8]:
def ct(file:str,chap_num:int,num:int,sub_num:int=None,engine:sqlalchemy.engine=engine)->None:
    """
    pubulic schema内のtableをすべて削除してから、指定のtable作成
    """
    if sub_num==None: 
        filename=f"../code-ssql2/ddl/ssql2-{file}{chap_num:02}_{num:02}"
    else:
        filename=f"../code-ssql2/ddl/ssql2-{file}{chap_num:02}_{num:02}_{sub_num}"
        
    with open(filename,"r") as f:
        stmt=f.read() 
    with engine.connect() as conn:
        conn.execute("drop schema public cascade; create schema public;")
        conn.execute(stmt)

In [9]:
def exe(stmt:str,engine:sqlalchemy.engine=engine):
    """
    sql実行
    """
    with engine.connect() as conn:
        query=conn.execute(stmt)
        try:
            result=query.fetchall()
            return result
        except:
            return None

In [14]:
def exe_df(stmt:str,engine:sqlalchemy.engine=engine)->pd.DataFrame:
    """
    sql(select)を実行してdataframeを返す
    """
    with engine.connect() as conn:
        df=pd.read_sql(stmt,conn)
    return df 

## List

In [15]:
ct("List",5,1)
exe_df("""
select 出金額,出金額+100,'SQL'
from 家計簿
""")

,出金額,?column?,?column?
0,380,480,SQL
1,0,100,SQL
2,2800,2900,SQL
3,5000,5100,SQL
4,7560,7660,SQL


In [16]:
ct("List",5,2)
exe_df("""
select 出金額,出金額+100 as increase
from 家計簿
""")

,出金額,increase
0,380,480
1,0,100
2,2800,2900
3,5000,5100
4,7560,7660


In [17]:
ct("List",5,3)
exe("""
insert into 家計簿
(出金額)
values (1000+105)
""")
exe("select * from 家計簿")

[(datetime.date(2018, 2, 3), '食費', 'コーヒーを購入', 0, 380),
 (datetime.date(2018, 2, 10), '給料', '1月の給料', 280000, 0),
 (datetime.date(2018, 2, 11), '教養娯楽費', '書籍を購入', 0, 2800),
 (datetime.date(2018, 2, 14), '交際費', '同期会の会費', 0, 5000),
 (datetime.date(2018, 2, 18), '水道光熱費', '1月の電気代', 0, 7560),
 (None, None, None, None, 1105)]

In [18]:
ct("List",5,3)
exe("""
update 家計簿
set 出金額=出金額+100
""")
exe("select * from 家計簿")

[(datetime.date(2018, 2, 3), '食費', 'コーヒーを購入', 0, 480),
 (datetime.date(2018, 2, 10), '給料', '1月の給料', 280000, 100),
 (datetime.date(2018, 2, 11), '教養娯楽費', '書籍を購入', 0, 2900),
 (datetime.date(2018, 2, 14), '交際費', '同期会の会費', 0, 5100),
 (datetime.date(2018, 2, 18), '水道光熱費', '1月の電気代', 0, 7660)]

## Q